In [1]:
#1 Import Necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import math
import stat
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

Problem Statement:
A cloth manufacturing company is interested to know about the segment or attributes causes high sale. 
Approach - A Random Forest can be built with target variable Sales (we will first convert it in categorical variable) & all other variable will be independent in the analysis.  

In [2]:
#2 Import Data
company_data = pd.read_csv('Company_Data (1).csv')
company_data

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


### DATA UNDERSTANDING

In [3]:
company_data.dtypes

Sales          float64
CompPrice        int64
Income           int64
Advertising      int64
Population       int64
Price            int64
ShelveLoc       object
Age              int64
Education        int64
Urban           object
US              object
dtype: object

In [4]:
company_data.isna().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

In [5]:
company_data.describe(include='all')

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400,400.000000,400.000000,400,400
unique,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,Medium,NaN,NaN,Yes,Yes
freq,NaN,NaN,NaN,NaN,NaN,NaN,219,NaN,NaN,282,258
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,NaN,53.322500,13.900000,NaN,NaN
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,NaN,16.200297,2.620528,NaN,NaN
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,NaN,25.000000,10.000000,NaN,NaN
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,NaN,39.750000,12.000000,NaN,NaN
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,NaN,54.500000,14.000000,NaN,NaN
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,NaN,66.000000,16.000000,NaN,NaN


In [6]:
company_data_copy= company_data.copy()
company_data_copy

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


**Data Transformation**
* OHE Encoding

In [7]:
company_data_ohe = pd.get_dummies(data=company_data_copy, columns = ['ShelveLoc','Urban','US'])
company_data_ohe

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes
0,9.50,138,73,11,276,120,42,17,1,0,0,0,1,0,1
1,11.22,111,48,16,260,83,65,10,0,1,0,0,1,0,1
2,10.06,113,35,10,269,80,59,12,0,0,1,0,1,0,1
3,7.40,117,100,4,466,97,55,14,0,0,1,0,1,0,1
4,4.15,141,64,3,340,128,38,13,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,33,14,0,1,0,0,1,0,1
396,6.14,139,23,3,37,120,55,11,0,0,1,1,0,0,1
397,7.41,162,26,12,368,159,40,18,0,0,1,0,1,0,1
398,5.94,100,79,7,284,95,50,12,1,0,0,0,1,0,1


### Model Building

In [8]:
x=company_data_ohe.drop(labels='Sales',axis=1)
y=company_data_ohe[['Sales']]

In [9]:
x

,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes
0,138,73,11,276,120,42,17,1,0,0,0,1,0,1
1,111,48,16,260,83,65,10,0,1,0,0,1,0,1
2,113,35,10,269,80,59,12,0,0,1,0,1,0,1
3,117,100,4,466,97,55,14,0,0,1,0,1,0,1
4,141,64,3,340,128,38,13,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,33,14,0,1,0,0,1,0,1
396,139,23,3,37,120,55,11,0,0,1,1,0,0,1
397,162,26,12,368,159,40,18,0,0,1,0,1,0,1
398,100,79,7,284,95,50,12,1,0,0,0,1,0,1


In [10]:
y

,Sales
0,9.50
1,11.22
2,10.06
3,7.40
4,4.15
...,...
395,12.57
396,6.14
397,7.41
398,5.94


In [11]:
y.mean()

Sales    7.496325
dtype: float64

In [12]:
import warnings
warnings.filterwarnings('ignore')

**Since the target variable is continious, we create a class of the value based on the mean <= 7.49 == "Min" and > 7.49 == "Max"**

In [13]:
for i in range(len(company_data_ohe['Sales'])):
    if company_data_ohe['Sales'][i] <= 7.49:
        company_data_ohe['Sales'][i] = 'Min'
    else:
        company_data_ohe['Sales'][i] = 'Max'

In [14]:
x=company_data_ohe.drop(labels='Sales',axis=1)
y=company_data_ohe[['Sales']]

In [15]:
x

,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes
0,138,73,11,276,120,42,17,1,0,0,0,1,0,1
1,111,48,16,260,83,65,10,0,1,0,0,1,0,1
2,113,35,10,269,80,59,12,0,0,1,0,1,0,1
3,117,100,4,466,97,55,14,0,0,1,0,1,0,1
4,141,64,3,340,128,38,13,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,33,14,0,1,0,0,1,0,1
396,139,23,3,37,120,55,11,0,0,1,1,0,0,1
397,162,26,12,368,159,40,18,0,0,1,0,1,0,1
398,100,79,7,284,95,50,12,1,0,0,0,1,0,1


In [16]:
y

,Sales
0,Max
1,Max
2,Max
3,Min
4,Min
...,...
395,Max
396,Min
397,Min
398,Min


In [81]:
# Saving feature names for later use
feature_list = list(company_data.columns)
feature_list

['Sales',
 'CompPrice',
 'Income',
 'Advertising',
 'Population',
 'Price',
 'ShelveLoc',
 'Age',
 'Education',
 'Urban',
 'US']

##### Train Test Split

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=12)

In [19]:
#Training data 
X_train.shape , y_train.shape

((320, 14), (320, 1))

In [20]:
#Test Data
X_test.shape , y_test.shape

((80, 14), (80, 1))

##### MODEL TRAINING

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)

RandomForestClassifier()

In [22]:
gb_model = GradientBoostingClassifier()
gb_model.fit(X_train,y_train)

GradientBoostingClassifier()

### GridSearch CV

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
grid_search = GridSearchCV(estimator=rf_model,param_grid = {'max_depth': [4,5,6,7,8],'criterion':['gini','entropy']},cv=5)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8]})

In [25]:
grid_search.best_params_

{'criterion': 'entropy', 'max_depth': 7}

In [26]:
grid_search.best_score_

0.79375

In [27]:
grid_search_gb = GridSearchCV(estimator=gb_model,param_grid = {'max_depth': [4,5,6,7,8]},cv=5)
grid_search_gb.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'max_depth': [4, 5, 6, 7, 8]})

In [28]:
grid_search_gb.best_params_

{'max_depth': 4}

In [29]:
grid_search_gb.best_score_

0.7875

### MODEL TRAINING

In [31]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='entropy',max_depth=7)  #by using Gridcv
rf_model.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=7)

In [32]:
gb_model = GradientBoostingClassifier(n_estimators=100,max_depth=4) #by using grid search cv
gb_model.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=4)

###### Model Testing with rf model

In [39]:
#Training data
y_train_pred_rf = rf_model.predict(X_train) 

In [40]:
#Test data
y_test_pred_rf = rf_model.predict(X_test) 

###### Model Testing with gb model

In [41]:
#Training data
y_train_pred_gb = gb_model.predict(X_train) 

In [42]:
#Test data
y_test_pred_gb = gb_model.predict(X_test) 

##### RF Model Evaluation

In [43]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,classification_report

In [44]:
#Training Data 
accuracy_score(y_train,y_train_pred_rf)

0.98125

In [45]:
confusion_matrix(y_train,y_train_pred_rf)

array([[154,   5],
       [  1, 160]], dtype=int64)

In [46]:
#TEST DATA
accuracy_score(y_test,y_test_pred_rf)

0.8

In [47]:
confusion_matrix(y_test,y_test_pred_rf)

array([[28, 12],
       [ 4, 36]], dtype=int64)

In [53]:
print(classification_report(y_train,y_train_pred_rf))

              precision    recall  f1-score   support

         Max       0.99      0.97      0.98       159
         Min       0.97      0.99      0.98       161

    accuracy                           0.98       320
   macro avg       0.98      0.98      0.98       320
weighted avg       0.98      0.98      0.98       320



In [66]:
print(classification_report(y_test,y_test_pred_rf))

              precision    recall  f1-score   support

         Max       0.88      0.70      0.78        40
         Min       0.75      0.90      0.82        40

    accuracy                           0.80        80
   macro avg       0.81      0.80      0.80        80
weighted avg       0.81      0.80      0.80        80



#### GB model evluation

In [48]:
#Training Data 
accuracy_score(y_train,y_train_pred_gb)

1.0

In [49]:
confusion_matrix(y_train,y_train_pred_gb)

array([[159,   0],
       [  0, 161]], dtype=int64)

In [50]:
#TEST DATA
accuracy_score(y_test,y_test_pred_gb)

0.8125

In [51]:
confusion_matrix(y_test,y_test_pred_gb)

array([[31,  9],
       [ 6, 34]], dtype=int64)

In [52]:
print(classification_report(y_train,y_train_pred_gb))

              precision    recall  f1-score   support

         Max       1.00      1.00      1.00       159
         Min       1.00      1.00      1.00       161

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320



In [65]:
print(classification_report(y_test,y_test_pred_gb))

              precision    recall  f1-score   support

         Max       0.84      0.78      0.81        40
         Min       0.79      0.85      0.82        40

    accuracy                           0.81        80
   macro avg       0.81      0.81      0.81        80
weighted avg       0.81      0.81      0.81        80



In [57]:
rf_model.feature_importances_

array([0.10867158, 0.08560359, 0.11526543, 0.07440959, 0.23889023,
       0.11713457, 0.04739668, 0.06970358, 0.07208704, 0.02404276,
       0.0090926 , 0.00698797, 0.0155542 , 0.01516018])

In [60]:
fi = pd.DataFrame({'feature': list(X_train.columns),
                   'importance': rf_model.feature_importances_}).\
                    sort_values('importance', ascending = False)

In [61]:
fi

,feature,importance
4,Price,0.238890
5,Age,0.117135
2,Advertising,0.115265
0,CompPrice,0.108672
1,Income,0.085604
3,Population,0.074410
8,ShelveLoc_Good,0.072087
7,ShelveLoc_Bad,0.069704
6,Education,0.047397
9,ShelveLoc_Medium,0.024043


**Hence Price is the most important feature**

**=====================================================================================================================**

In [62]:
gb_model.feature_importances_

array([0.15448275, 0.0742758 , 0.1252573 , 0.02711716, 0.30971086,
       0.09607839, 0.01955307, 0.06598714, 0.11376487, 0.00714993,
       0.00136827, 0.00146278, 0.00109476, 0.00269692])

In [63]:
fi_gb = pd.DataFrame({'feature': list(X_train.columns),
                   'importance': gb_model.feature_importances_}).\
                    sort_values('importance', ascending = False)

In [103]:
fi_gb

,feature,importance
4,Price,0.309711
0,CompPrice,0.154483
2,Advertising,0.125257
8,ShelveLoc_Good,0.113765
5,Age,0.096078
1,Income,0.074276
7,ShelveLoc_Bad,0.065987
3,Population,0.027117
6,Education,0.019553
9,ShelveLoc_Medium,0.007150


**Hence Price is the most important feature for both RF and GB model**